In [ ]:
import os
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Concatenate
from keras.layers import GlobalAveragePooling2D, Dense, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
# Data preparation
dataset_dir= 'TNB_split'
input_shape=(224, 224, 3)
batch_size=8
num_classes=5

In [ ]:
#Data augmentation
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen=ImageDataGenerator(
    rescale=1./255)

In [ ]:
#Generate the train dataset, validation dataset and test dataset
train_generator=train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical', #skin has two classes: benign and malignant
    )

validation_generator=train_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'val'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical', #skin has two classes: benign and malignant
    )

test_generator=test_datagen.flow_from_directory(
    os.path.join(dataset_dir, 'test'),
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical', #skin has two classes: benign and malignant
    )

In [ ]:
def inception_block(inputs, filters):
    branch1 = Conv2D(filters[0], (1, 1), padding='same', activation='relu')(inputs)
    branch1 = Conv2D(filters[1], (3, 3), padding='same', activation='relu')(branch1)

    branch2 = Conv2D(filters[2], (1, 1), padding='same', activation='relu')(inputs)
    branch2 = Conv2D(filters[3], (5, 5), padding='same', activation='relu')(branch2)

    branch3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch3 = Conv2D(filters[4], (1, 1), padding='same', activation='relu')(branch3)

    branch4 = Conv2D(filters[5], (1, 1), padding='same', activation='relu')(inputs)

    output = Concatenate(axis=-1)([branch1, branch2, branch3, branch4])
    return output

In [ ]:
def InceptionModel(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial Convolution layer
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)

    # Inception Blocks
    x = inception_block(x, [16, 32, 8, 16, 8, 16])
    x = inception_block(x, [32, 64, 16, 32, 16, 32])
    x = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    x = inception_block(x, [64, 128, 32, 64, 32, 64])
    x = inception_block(x, [128, 256, 64, 128, 64, 128])
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)

    outputs = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
model = InceptionModel(input_shape, num_classes)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=2, validation_data=validation_generator, verbose=1)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training', 'Validation'], loc='upper right')
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

In [ ]:
from keras import backend as K
import gc

In [ ]:
K.clear_session()
gc.collect()

In [ ]:
del model

In [ ]:
# you will need to install numba using "pip install numba"
from numba import cuda

In [ ]:
cuda.select_device(0)
cuda.close()